In [4]:
from bread.algo.lineage import LineageGuesserBudLum, LineageGuesserExpansionSpeed, LineageGuesserMinDistance, LineageGuesserMinTheta, accuracy, NotEnoughFramesException
from bread.algo.lineage.nn import LineageGuesserDeepNN
from bread.data import Lineage, Segmentation, Microscopy, BreadWarning

In [8]:
from utils import Timer
import multiprocessing
import warnings
import pandas

In [6]:
accs = { 'BudLum': [], 'ExpansionSpeed': [], 'MinTheta': [], 'MinDistance': [], 'DeepNN': [] }
colony_ids = [1,2,3,4,5]

def process(colony_id: int):
	with Timer(f'load data #{colony_id}'):
		seg = Segmentation.from_h5(f'../../data/raw/colony{colony_id:03d}_segmentation.h5')
		gfp = Microscopy.from_tiff(f'../../data/raw/colony{colony_id:03d}_GFP.tif')
		lin_truth = Lineage.from_csv(f'../../data/raw/colony{colony_id:03d}_lineage.csv')

	with warnings.catch_warnings():
		warnings.filterwarnings('ignore', category=NotEnoughFramesException)
		with Timer(f'guess lineage #{colony_id} (BudLum)'):
			lin_pred_budlum = LineageGuesserBudLum(budneck_img=gfp, segmentation=seg).guess_lineage()
		with Timer(f'guess lineage #{colony_id} (ExpSpeed)'):
			lin_pred_expspeed = LineageGuesserExpansionSpeed(segmentation=seg).guess_lineage()
		with Timer(f'guess lineage #{colony_id} (MinTheta)'):
			lin_pred_mintheta = LineageGuesserMinTheta(segmentation=seg).guess_lineage()
		with Timer(f'guess lineage #{colony_id} (MinDistance)'):
			lin_pred_mindist = LineageGuesserMinDistance(segmentation=seg).guess_lineage()
		with Timer(f'guess lineage #{colony_id} (NN)'):
			lin_pred_nn = LineageGuesserDeepNN(segmentation=seg,
				scale_length=1/9.2308, scale_time=5,
				path_model='../../pytorch/best/best.ckpt'
			).guess_lineage()
	
	return (
		accuracy(lin_truth, lin_pred_budlum, strict=False),
		accuracy(lin_truth, lin_pred_expspeed, strict=False),
		accuracy(lin_truth, lin_pred_mintheta, strict=False),
		accuracy(lin_truth, lin_pred_mindist, strict=False),
		accuracy(lin_truth, lin_pred_nn, strict=False),
	)

with multiprocessing.Pool() as pool:
	results = pool.imap(process, colony_ids)

	for res in results:
		for key, a in zip(accs, res):
			accs[key].append(a)

"load data #5" finished in 0.88s
"load data #3" finished in 1.45s
"load data #4" finished in 1.61s
"load data #1" finished in 1.67s


/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/algo/lineage/_lineage.py:204: BreadWarning: cell #3 does not have nearest neighbours with a distance less than 8, and flexible_threshold is False.
  warnings.warn(BreadWarning(f'cell #{nearest_neighbours_of} does not have nearest neighbours with a distance less than {self.nn_threshold}, and flexible_threshold is {self.flexible_nn_threshold}.'))
/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/algo/lineage/_lineage.py:264: LineageWarning: Unable to determine parent, got exception NoCandidateParentException('No candidate parents have been found for in frame #19.'). Computation for frame #19, bud #3 skipped.
  warnings.warn(LineageWarning(f'Unable to determine parent, got exception {repr(e)}. Computation for frame #{time_id_}, bud #{bud_id} skipped.'))
/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/algo/lineage/_lineage.py:

"guess lineage #5 (BudLum)" finished in 1.93s
"load data #2" finished in 4.36s


/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/algo/lineage/_lineage.py:516: NotEnoughFramesWarning: Not enough frames in the movie (bud #38 at frame #178), requested 5, but only 3 remaining.
  warnings.warn(NotEnoughFramesWarning(bud_id, time_id, self.num_frames, len(frame_range)))


"guess lineage #5 (ExpSpeed)" finished in 1.72s


...

"guess lineage #2 (BudLum)" finished in 33.59s
"guess lineage #4 (MinDistance)" finished in 3.04s


/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/data/_features.py:81: RuntimeWarning: Mean of empty slice
  expspeed = np.nanmean(np.gradient(expdists))
/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/data/_features.py:81: RuntimeWarning: Mean of empty slice
  expspeed = np.nanmean(np.gradient(expdists))
/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/data/_features.py:81: RuntimeWarning: Mean of empty slice
  expspeed = np.nanmean(np.gradient(expdists))
/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/data/_features.py:81: RuntimeWarning: Mean of empty slice
  expspeed = np.nanmean(np.gradient(expdists))


"guess lineage #4 (NN)" finished in 4.12s


/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/algo/lineage/_lineage.py:516: NotEnoughFramesWarning: Not enough frames in the movie (bud #150 at frame #178), requested 5, but only 3 remaining.
  warnings.warn(NotEnoughFramesWarning(bud_id, time_id, self.num_frames, len(frame_range)))
/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/algo/lineage/_lineage.py:516: NotEnoughFramesWarning: Not enough frames in the movie (bud #151 at frame #178), requested 5, but only 3 remaining.
  warnings.warn(NotEnoughFramesWarning(bud_id, time_id, self.num_frames, len(frame_range)))
/run/media/nicole/nicole_data/Documents/2018 EPFL/PH MA 2/TP4/bread-git/bread/src/bread/algo/lineage/_lineage.py:516: NotEnoughFramesWarning: Not enough frames in the movie (bud #152 at frame #179), requested 5, but only 2 remaining.
  warnings.warn(NotEnoughFramesWarning(bud_id, time_id, self.num_frames, len(frame_range)))
/run/media/nicole/nico

"guess lineage #2 (ExpSpeed)" finished in 14.68s
"guess lineage #2 (MinTheta)" finished in 24.67s
"guess lineage #2 (MinDistance)" finished in 7.71s
"guess lineage #2 (NN)" finished in 10.09s


In [14]:
accs_df = pandas.DataFrame(accs, [f'colony {i}' for i in colony_ids], accs.keys())
accs_df

,BudLum,ExpansionSpeed,MinTheta,MinDistance,NN
colony 1,1.000000,0.750000,0.800000,0.762500,0.725000
colony 2,0.993243,0.806897,0.736486,0.743243,0.786207
colony 3,0.989474,0.731183,0.684211,0.610526,0.752688
colony 4,0.968000,0.650000,0.592000,0.656000,0.733333
colony 5,1.000000,0.805556,0.694444,0.777778,0.805556


In [13]:
accs_df.mean()

BudLum            0.990143
ExpansionSpeed    0.748727
MinTheta          0.701428
MinDistance       0.710009
NN                0.760557
dtype: float64